In [11]:
import re
import argparse
import os

In [8]:
with open('../pipeline/Snakefile_bak') as f:
    file = f.read()

In [9]:
file = re.sub('{MIN_ORF_LENGTH}', '300', file)
file = re.sub('{MIN_INT_LENGTH}', '100', file)
file = re.sub('{SEQ_IDENTITY}', '0.75', file)
file = re.sub('{SEQ_LEN_CUTOFF}', '0.75', file)

In [19]:
this is the default cd-hit's "global sequence identity" calculated as:
# 								   # number of identical amino acids in alignment
# 								   # divided by the full length of the shorter sequence

'\n\nrule get_orf:\n\tinput: "data/Sclerotinia_sclerotiorum.ASM14694v1.dna.toplevel.fa"\n\toutput: "data/Scler_predicted_orfs.fasta"\n\tshell: "getorf -sequence {input} -outseq {output} -find 3"\n\nrule get_frame_shifts:\n\tinput:\n\t\tdna = "data/Scler_predicted_orfs.fasta",\n\t\tgenome = "data/Sclerotinia_sclerotiorum.ASM14694v1.dna.toplevel.fa"\n\toutput: "data/Scler_300_100.txt"\n\trun:\n\t\tshell("python3 scripts/Frshft_FINDER.py -input_dna {input.dna} -input_genome {input.genome} -min_orf 300 -interval_length 100 -output {output}")\n\nrule get_intersected_prot:\n\tinput: "data/Scler_300_100.txt"\n\toutput: "data/Scler_300_100_intersected_prot.fasta"\n\tshell: "grep -i \'PROT_INTER\' -A1 {input} | grep -v \'\\-\\-\' > {output}"\n\nrule cdhit_filter:\n\tinput: "data/Scler_300_100_intersected_prot.fasta"\n\toutput: "data/Scler_300_100_cdhit_filtred.fasta"\n\tshell: "cdhit -i {input} -o {output} -c 0.75 -s 0.75"\n'

In [25]:
import re
import argparse
import os

def main(args):

    with open(args.pipeline) as f:
        file = f.read()
    
    file = re.sub('{GENOME}', args.genome, file)
    file = re.sub('{DATA_PATH}', os.path.dirname(args.genome)+"/", file)
    file = re.sub('{SCRIPTS_PATH}', args.scripts_path.rstrip('/')+'/', file)
    file = re.sub('{PREFIX}', args.prefix, file)
    file = re.sub('{MIN_ORF_LENGTH}', args.min_orf, file)
    file = re.sub('{MIN_INT_LENGTH}', args.min_int, file)
    file = re.sub('{SEQ_IDENTITY}', args.seq_I, file)
    file = re.sub('{SEQ_LEN_CUTOFF}', args.seq_L, file)
    
    print(file)
    
if __name__=='__main__':
    
#     parser = argparse.ArgumentParser(
#         formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    
#     parser.add_argument('-pipeline', type=str, help='The path to the input genome.')
#     parser.add_argument('-genome', type=str, help='The path to the input genome.')
#     parser.add_argument('-prefix', type=str, help='Additional prefix for your files with delimeter f.e. "hg19_".')
#     parser.add_argument('-min_orf', type=str, default='300', help='Min ORF length.')
#     parser.add_argument('-min_int', type=str, default='100', help='Min interval length')
#     parser.add_argument('-seq_I', type=str, default='0.75', help="This is the default cd-hit's 'global sequence identity' calculated as:\nnumber of identical amino acids in alignment\ndivided by the full length of the shorter sequence.")
#     parser.add_argument('-seq_L', type=str, default='0.75', help="this is the default cd-hit's 'length identity cutoff',\nf.e., if set to 0.9, the shorter sequences need to be\nat least 90% length of the representative of the cluster")
#     args = parser.parse_args()
    
    class Args():
        def __init__(self):
            self.pipeline = '../pipeline/Snakefile.template'
            self.genome = '../pipeline/data/Sclerotinia_sclerotiorum.ASM14694v1.dna.toplevel.fa'
            self.scripts_path = '../pipeline/scripts/'
            self.prefix = 'Scler_'
            self.min_orf = '300'
            self.min_int = '100'
            self.seq_I = '0.75'
            self.seq_L = '0.75'
    args = Args()
    main(args)
    



rule get_orf:
	input: "../pipeline/data/Sclerotinia_sclerotiorum.ASM14694v1.dna.toplevel.fa"
	output: "../pipeline/data/Scler_predicted_orfs.fasta"
	shell: "getorf -sequence {input} -outseq {output} -find 3"

rule get_frame_shifts:
	input:
		dna = "../pipeline/data/Scler_predicted_orfs.fasta",
		genome = "../pipeline/data/Sclerotinia_sclerotiorum.ASM14694v1.dna.toplevel.fa"
	output: "../pipeline/data/Scler_300_100.txt"
	run:
		shell(f"python3 ../pipeline/scripts/Frshft_FINDER.py -input_dna {input.dna} -input_genome {input.genome} -min_orf 300 -interval_length 100 -output {output}")

rule get_intersected_prot:
	input: "../pipeline/data/Scler_300_100.txt"
	output: "../pipeline/data/Scler_300_100_intersected_prot.fasta"
	shell: "grep -i 'PROT_INTER' -A1 {input} | grep -v '\-\-' > {output}"

rule cdhit_filter:
	input: "../pipeline/data/Scler_300_100_intersected_prot.fasta"
	output: "../pipeline/data/Scler_300_100_cdhit_filtred.fasta"
	shell: "cdhit -i {input} -o {output} -c 0.75 -s 0.75"

In [20]:
os.path.dirname('../pipeline/data/Sclerotinia_sclerotiorum.ASM14694v1.dna.toplevel.fa')

'../pipeline/data'